<a href="https://colab.research.google.com/github/JunsolKim/factchecker_bias/blob/main/politifact_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My Drive

/content/drive/My Drive


In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def crawler(i):
    url = 'https://www.politifact.com/factchecks/?page='+str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    results = []
    for article in soup.select('article.m-statement.m-statement--is-medium'):
        url_sub = 'https://www.politifact.com' + article.select('a')[1]['href']
        page_sub = requests.get(url_sub)
        soup_sub = BeautifulSoup(page_sub.content, 'lxml')
        
        name = soup_sub.find(class_='m-statement__name')
        name_str = name.get_text().strip()
        name_link = 'https://www.politifact.com' + name['href']
        statement_str = soup_sub.find(class_='m-statement__quote').get_text
        rating_link = soup_sub.select('picture')[1].select('img')[0]['src']

        when_where_str = soup_sub.find(class_='m-statement__desc').get_text().strip()

        title_str = soup_sub.select('h2.c-title.c-title--subline')[0].get_text().strip()
        content_str = soup_sub.find(class_='m-textblock').get_text().strip()

        factchecker = soup_sub.find(class_='m-author__content').select('a')[0]
        factchecker_str = factchecker.get_text().strip()
        factchecker_link = 'https://www.politifact.com' + factchecker['href']
        factcheck_date = soup_sub.find(class_='m-author__date').get_text().strip()

        results.append([url_sub, 
                        name_str, 
                        name_link, 
                        statement_str, 
                        rating_link, 
                        when_where_str, 
                        title_str, 
                        content_str, 
                        factchecker_str, 
                        factchecker_link, 
                        factcheck_date])
    return results

executor = ThreadPoolExecutor(8)
results_all = list(tqdm(executor.map(crawler, range(1,699)), total=698)) #1, 699

In [11]:
from functools import reduce
from operator import iconcat 
import pandas as pd

df = pd.DataFrame(reduce(iconcat, results_all), columns=['url', 
                                                         'speaker', 
                                                         'speaker_info', 
                                                         'statement', 
                                                         'rating', 
                                                         'where_when', 
                                                         'title', 
                                                         'content', 
                                                         'factchecker', 
                                                         'factchecker_info', 
                                                         'factcheck_date'])

df.to_csv('politifact.csv')

,url,speaker,speaker_info,where_when,title,factchecker,factchecker_info,factcheck_date
0,https://www.politifact.com/factchecks/2022/mar...,Facebook posts,https://www.politifact.com/personalities/faceb...,"stated on March 6, 2022 in a Facebook post:",Fauci has regularly given interviews and appea...,Ciara O'Rourke,https://www.politifact.com/staff/ciara-orourke/,"March 7, 2022"
1,https://www.politifact.com/factchecks/2022/mar...,Evan McMullin,https://www.politifact.com/personalities/evan-...,"stated on March 6, 2022 in an ad:",Evan McMullin ad portraying GOP Utah Sen. Mike...,Tom Kertscher,https://www.politifact.com/staff/tom-kertscher/,"March 7, 2022"
2,https://www.politifact.com/factchecks/2022/mar...,Facebook posts,https://www.politifact.com/personalities/faceb...,"stated on March 1, 2022 in a post:",Claim misleads about 2020 press conference orc...,Monique Curet,https://www.politifact.com/staff/monique-curet/,"March 7, 2022"
3,https://www.politifact.com/factchecks/2022/mar...,Viral image,https://www.politifact.com/personalities/viral...,"stated on March 4, 2022 in a Facebook post:","California gas prices are going up, but not at...",Ciara O'Rourke,https://www.politifact.com/staff/ciara-orourke/,"March 7, 2022"
4,https://www.politifact.com/factchecks/2022/mar...,Jamaal Bowman,https://www.politifact.com/personalities/jamaa...,"stated on January 26, 2022 in a tweet:",Fact-checking Rep. Jamaal Bowman on legacy adm...,Elizabeth Egan,https://www.politifact.com/staff/elizabeth-egan/,"March 7, 2022"
...,...,...,...,...,...,...,...,...
385,https://www.politifact.com/factchecks/2022/feb...,Facebook posts,https://www.politifact.com/personalities/faceb...,"stated on February 12, 2022 in a post:","Image is from Woodstock festival in 1969, not ...",Monique Curet,https://www.politifact.com/staff/monique-curet/,"February 15, 2022"
386,https://www.politifact.com/factchecks/2022/feb...,Facebook posts,https://www.politifact.com/personalities/faceb...,"stated on February 9, 2022 in a post:","Footage is from video game, does not show actu...",Monique Curet,https://www.politifact.com/staff/monique-curet/,"February 15, 2022"
387,https://www.politifact.com/factchecks/2022/feb...,Instagram posts,https://www.politifact.com/personalities/insta...,"stated on January 22, 2022 in a post:","No, this 9-year-old was not arrested at a NYC ...",Monique Curet,https://www.politifact.com/staff/monique-curet/,"February 15, 2022"
388,https://www.politifact.com/factchecks/2022/feb...,Chip Roy,https://www.politifact.com/personalities/chip-...,"stated on February 1, 2022 in a tweet:",Pfizer’s data on COVID-19 vaccine for children...,Nusaiba Mizan,https://www.politifact.com/staff/nusaiba-mizan/,"February 15, 2022"


In [16]:
a[29]

<article class="m-statement m-statement--is-medium m-statement--false">
<div class="m-statement__author">
<div class="m-statement__avatar">
<div class="m-statement__image">
<div class="c-image" style="padding-top: 119.27710843373494%;">
<img class="c-image__thumb" height="99" src="https://static.politifact.com/CACHE/images/politifact/mugs/Facebook_Logo_2019/3b3162a610bc8b3cbeaa2f750c7b6868.jpg" width="83"/>
<picture>
<img class="c-image__original " height="178" src="https://static.politifact.com/CACHE/images/politifact/mugs/Facebook_Logo_2019/7cf81e6f294db8af7f47a3cb47f42fc2.jpg" width="166"/>
</picture>
</div>
</div>
</div>
<div class="m-statement__meta">
<a class="m-statement__name" href="/personalities/facebook-posts/" title="Facebook posts">
Facebook posts
</a>
<div class="m-statement__desc">
stated on February 25, 2022 in a post:
</div>
</div>
</div>
<div class="m-statement__content">
<div class="m-statement__body">
<div class="m-statement__quote-wrap">
<div class="m-statement__qu

In [4]:
soup

<!DOCTYPE html>
<html dir="ltr" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title> | PolitiFact </title>
<meta content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." name="description"/>
<meta content="PolitiFact" name="twitter:username"/>
<meta content="summary" name="twitter:card"/>
<meta content="PolitiFact" name="twitter:site"/>
<meta content="" name="twitter:url"/>
<meta content="" name="twitter:title"/>
<meta content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." name="twitter:description"/>
<meta content="" name="twitter:image"/>
<link href="https://static.politifact.com/images/favicon.ico" rel="shortcut icon" type="image/ico"/>
<link href="https://static.politifact.com/images/favicon.png" rel="

In [ ]:


def crawler(statement):
    try:
        page = requests.get("https://www.politifact.com/search/?q="+statement[:30].replace(' ','+'))
        #name = soup.find(class_='o-listease__item').find('a').get_text().lower()
        #first_name = name[:name.index(' ')]
        checker = soup.find(class_='o-listease__item').find(class_='c-textgroup__meta').get_text()
        checker_name = checker_name[3:checker_name.index('•')]
        checker_name = checker.strip()
        return checker_name
    except:
        return ""

executor = ThreadPoolExecutor(25)
name_list = list(tqdm(executor.map(crawler, list(df['statement'])), total=len(df['statement'])))


In [ ]:
!wget liar_dataset.zip https://www.cs.ucsb.edu/~william/data/liar_dataset.zip

--2022-01-05 12:29:40--  http://liar_dataset.zip/
Resolving liar_dataset.zip (liar_dataset.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘liar_dataset.zip’
--2022-01-05 12:29:40--  https://www.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving www.cs.ucsb.edu (www.cs.ucsb.edu)... 23.185.0.253, 2620:12a:8001::253, 2620:12a:8000::253
Connecting to www.cs.ucsb.edu (www.cs.ucsb.edu)|23.185.0.253|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip [following]
--2022-01-05 12:29:40--  https://sites.cs.ucsb.edu/~william/data/liar_dataset.zip
Resolving sites.cs.ucsb.edu (sites.cs.ucsb.edu)... 128.111.27.164
Connecting to sites.cs.ucsb.edu (sites.cs.ucsb.edu)|128.111.27.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1013571 (990K) [application/zip]
Saving to: ‘liar_dataset.zip’

liar_dataset.zip    100%[===================>] 989.82K  

In [ ]:
!unzip liar_dataset.zip

Archive:  liar_dataset.zip
  inflating: README                  
  inflating: test.tsv                
  inflating: train.tsv               
  inflating: valid.tsv               


In [ ]:
import pandas as pd
df = pd.read_csv('train.tsv', delimiter="\t", header=None, names=['id', 
                                                                  'label', 
                                                                  'statement', 
                                                                  'subject', 
                                                                  'speaker', 
                                                                  'job_title', 
                                                                  'state_info', 
                                                                  'party', 
                                                                  'barely_true', 
                                                                  'false', 
                                                                  'half_true', 
                                                                  'mostly_true', 
                                                                  'pants_on_fire', 
                                                                  'context'])

Crawl fact-checker names

In [ ]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

def crawler(statement):
    try:
        page = requests.get("https://www.politifact.com/search/?q="+statement[:30].replace(' ','+'))
        soup = BeautifulSoup(page.content, 'lxml')
        #name = soup.find(class_='o-listease__item').find('a').get_text().lower()
        #first_name = name[:name.index(' ')]
        checker = soup.find(class_='o-listease__item').find(class_='c-textgroup__meta').get_text()
        checker_name = checker_name[3:checker_name.index('•')]
        checker_name = checker.strip()
        return checker_name
    except:
        return ""

executor = ThreadPoolExecutor(25)
name_list = list(tqdm(executor.map(crawler, list(df['statement'])), total=len(df['statement'])))

100%|██████████| 10240/10240 [26:29<00:00,  6.44it/s]


In [ ]:
df['rated_by'] = name_list

In [ ]:
from tqdm.auto import tqdm

a = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    count = 1
    for name in str(df.loc[index, 'rated_by']).split(', '):
        df.loc[index, 'rated_by_'+str(count)] = name
        count += 1

In [ ]:
df.to_csv('train_2.csv')